In [ ]:
!pip install google-cloud-bigquery pandas-gbq

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#### Querying from the Products table:

In [ ]:
import pandas as pd
from google.cloud import bigquery
from google.colab import auth

auth.authenticate_user()

project_id = 'querytest2-444200'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Query the public dataset
query = """
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.products`
    LIMIT 10000
"""

# Run the query
query_job = client.query(query)

# Get the results
results = query_job.result()

# Convert to a Pandas DataFrame
products = results.to_dataframe()

products.head()

#### Data Cleaning

Confirming that the number of ID are unique for each row.

In [ ]:
products = products_and_orders
products['id'].nunique()

10000

We are missing some values from the `name`, and `brand` columns.

In [ ]:
products.isnull().sum()

,0
id,0
cost,0
category,0
name,0
brand,24
retail_price,0
department,0
sku,0
distribution_center_id,0


In [ ]:
products.duplicated().sum()

0

In [ ]:
products.columns.duplicated().sum()

0

This quick overview informs us that the data is relatively clean and there are no duplicates rows or columns. Let's address those missing values in the `name` and `brand` columns:

In [ ]:
products[products['name'].isna()]

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
3247,12586,18.972000,Intimates,None,Josie by Natori,36.000000,Women,A7EA034186E14FB5F7B37CF664893CD2,1
5588,24455,67.335453,Outerwear & Coats,None,Tru-Spec,147.990005,Men,B290A635641F585B3DD6B95FD42DC267,2


For the missing values in the name column, we see that there are no other values missing from other columns. Let's imput the missing names with 'unknown'.

In [ ]:
products['name'].fillna('unknown', inplace=True)
products.isna().sum()

,0
id,0
cost,0
category,0
name,0
brand,24
retail_price,0
department,0
sku,0
distribution_center_id,0


Addressing missing values from `brand` column:

In [ ]:
missingbrands = products['brand'].isna().sum()
print(f'The number of missing values from the brand column is {missingbrands}')
products[products['brand'].isna()].head()

The number of missing values from the brand column is 24


,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
7820,755,15.140850,Tops & Tees,The Very Hungry Caterpillar Classic Book Women...,None,28.950001,Women,CCB0989662211F61EDAE2E26D58EA92F,3
7821,1629,23.572350,Fashion Hoodies & Sweatshirts,Carhartt Women's Hooded Knit Jacket,None,45.950001,Women,5C50B4DF4B176845CD235B6A510C6903,3
7822,8600,16.015551,Outerwear & Coats,Women's Micro Fleece Vest (up to size 4X),None,35.990002,Women,CE840AA9583592E71F3DB26EE6E41703,3
7823,9482,5.712000,Socks & Hosiery,KEEN Women Bellingham Low Ultralite Sock,None,16.000000,Women,C5A3C867A3DFB776552DCFC804D4937E,3
7824,10598,9.616400,Intimates,JMS Comfort Lace with Hidden Shaper,None,16.580000,Women,E6FA05C07B144B6FF4D1FD1CD2BBB244,3


Peeping into these row, we can see that the rows with missing brand values have a lot of the brand names inside the name description. We could potentially extract the names of the brand from the name column, however, given that this is not true for all rows, and that the length of the strings vary, it would be hard to automate the custom processes that we would create for this data sample.  Most importantly, it would interfere with our data pipelines. We will impute the brand as unknown instead.  


In [ ]:
products['brand'].fillna({'brand':'unknown'}, inplace=True)
products.isna().sum()

,0
id,0
cost,0
category,0
name,0
brand,0
retail_price,0
department,0
sku,0
distribution_center_id,0


We have successfully assessed duplicates, and missing values. Our data is cleaned.

**Logistic checkpoint: Saving cleaned datatset**

In [ ]:
productscsv = products.to_csv(path_or_buf='/content/drive/MyDrive/2024BusinessAnalysisProject/products.csv')